In [216]:
import pandas as pd
import numpy as np
import openpyxl
from datetime import datetime
from dateutil.relativedelta import relativedelta
import gc

In [217]:
file_path = "HIST2.csv"

In [218]:
import csv

data_rows = []
# Open and read the CSV file
with open(file_path, newline='', encoding='utf-8') as file:
    reader = csv.reader(file)
    
    # Loop through each row and check if any cell contains '----'
    # for row in reader:
    #     if any('----' in cell for cell in row):
    #         print(row)  # Print the row containing the pattern

    i = 0
    for row in reader:
        i+= 1
        if "Historical Data Set" in row[1]:
            data_rows.append(i)
            # print(row)
            # print(i)
            #break
        
    print(data_rows)

[24, 148, 272, 396, 520, 644, 768, 892, 1016, 1140, 1264, 1388]


In [219]:
differences = [data_rows[i+1] - data_rows[i] for i in range(len(data_rows)-1)]
if len(set(differences)) == 1:
    print("All differences are the same")


All differences are the same


In [220]:
set(differences)

{124}

In [221]:
# Read the CSV file, skipping the first 24 rows
df = pd.read_csv(file_path, skiprows=range(data_rows[0]-1))

new_df = pd.DataFrame(columns=['billing_period','cumulative_import', 'cumulative_export','apparent_import', 'reactive_import',
                               'reactive_export','rate1', 'rate2', 'rate3', 'rate4', 'rate5', 
        'rate6','max_dem1','max_dem1_datetime','max_dem2','max_dem2_datetime','max_dem3','max_dem3_datetime',
        ])

# read rows between  24 and 24 + differences[0]
for x in data_rows:
    df1 = pd.read_csv(file_path, skiprows=range(x-1), nrows=differences[0])
    df1 = df1[[df.columns[3],df1.columns[4],df1.columns[5],df1.columns[6]]]
    df1 = df1.rename(columns={df1.columns[0]:'column1',df1.columns[1]:'column2',df1.columns[2]:'column3',df1.columns[3]:'column4'})
    

    first_index_imp = df1[df1['column2'] == 'Import Wh'].index[0] 
    cumulative_import = float(df1.at[first_index_imp, 'column1'].replace(",",""))/1000

    first_index_exp = df1[df1['column2'] == 'Import Wh'].index[0] 
    cumulative_export = float(df1.at[first_index_exp, 'column1'].replace(",",""))/1000

    apparent_import_index = df1[df1['column2'] == 'Total VAh'].index[0] 
    apparent_import = float(df1.at[apparent_import_index, 'column1'].replace(",",""))/1000

    first_Q_react_imp = df1[df1['column2'] == 'Q1 varh'].index[0]
    second_Q_react_imp = df1[df1['column2'] == 'Q2 varh'].index[0]
    reactive_import = float(df1.at[first_Q_react_imp, 'column1'].replace(",",""))/1000 + float(df1.at[second_Q_react_imp, 'column1'].replace(",",""))/1000

    first_Q_react_exp = df1[df1['column2'] == 'Q3 varh'].index[0]
    second_Q_react_exp = df1[df1['column2'] == 'Q4 varh'].index[0]
    reactive_export = float(df1.at[first_Q_react_exp, 'column1'].replace(",",""))/1000 + float(df1.at[second_Q_react_exp, 'column1'].replace(",",""))/1000

    rate1 = float(df1.loc[df1['column1'] == 'Rate 1', 'column2'].iloc[0].replace(",",""))/1000
    rate2 = float(df1.loc[df1['column1'] == 'Rate 2', 'column2'].iloc[0].replace(",",""))/1000
    rate3 = float(df1.loc[df1['column1'] == 'Rate 3', 'column2'].iloc[0].replace(",",""))/1000

    rate4 = float(df1.loc[df1['column1'] == 'Rate 4', 'column2'].iloc[0].replace(",",""))/1000
    rate5 = float(df1.loc[df1['column1'] == 'Rate 5', 'column2'].iloc[0].replace(",",""))/1000
    rate6 = float(df1.loc[df1['column1'] == 'Rate 6', 'column2'].iloc[0].replace(",",""))/1000
    
    ## MAXIMUM DEMANDS
    max_dem1 = df1.loc[df1['column1'] == 'MD 1a', 'column2'].iloc[0].replace(",","")
    max_dem1_time = df1.loc[df1['column1'] == 'MD 1a', 'column4'].iloc[0]

    ## MAXIMUM DEMANDS
    max_dem2 = df1.loc[df1['column1'] == 'MD 2a', 'column2'].iloc[0].replace(",","")
    max_dem2_time = df1.loc[df1['column1'] == 'MD 2a', 'column4'].iloc[0]

    max_dem3 = df1.loc[df1['column1'] == 'MD 3a', 'column2'].iloc[0].replace(",","")
    max_dem3_time = df1.loc[df1['column1'] == 'MD 3a', 'column4'].iloc[0]

    #COUNT
    billing_count = df1.loc[df1['column1'] == 'Count', 'column2'].iloc[0]

    #BILLING PERIOD
    billing_end = df1.loc[df1['column1'] == 'Period End', 'column2'].iloc[0]

    dt1 = datetime.strptime(max_dem1_time, "%A, %B %d, %Y %I:%M:%S %p")
    max_dem1_time = dt1.strftime("%Y-%m-%d %H:%M")

    dt2 = datetime.strptime(max_dem2_time, "%A, %B %d, %Y %I:%M:%S %p")
    max_dem2_time = dt2.strftime("%Y-%m-%d %H:%M")

    dt3 = datetime.strptime(max_dem3_time, "%A, %B %d, %Y %I:%M:%S %p")
    max_dem3_time = dt3.strftime("%Y-%m-%d %H:%M")

    dt = datetime.strptime(billing_end, "%A, %B %d, %Y %I:%M %p")

    if dt.day == 1:
        billing_period = dt - relativedelta(months=1)
        billing_period = billing_period.strftime("%Y-%m-%d")

    else:
        billing_period = dt.strftime("%Y-%m-%d")

    row1 = pd.DataFrame({
        'billing_period': [billing_period],
        'cumulative_import': [cumulative_import],
        'cumulative_export': [cumulative_export],
        'apparent_import':[apparent_import],
        'reactive_import':[reactive_import],
        'reactive_export':[reactive_export],
        'rate1': [rate1],
        'rate2': [rate2],
        'rate3': [rate3],
        'rate4': [rate4],
        'rate5': [rate5],
        'rate6': [rate6],
        'no_of_resets':[billing_count],
        'max_dem1':[max_dem1],
        'max_dem1_datetime':[max_dem1_time],
        'max_dem2':[max_dem2],
        'max_dem2_datetime':[max_dem2_time],
        'max_dem3':[max_dem3],
        'max_dem3_datetime':[max_dem3_time]
    })

    new_df = pd.concat([new_df, row1], ignore_index = True)

    columns_to_assign_na = ['vt_ratio', 'ct_ratio', 'date_of_last_reset', 'programing_count']

    for column in columns_to_assign_na:
        new_df[column] = pd.NA


C:\Users\HP\AppData\Local\Temp\ipykernel_1520\2708137643.py:98: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  new_df = pd.concat([new_df, row1], ignore_index = True)


In [ ]:
new_df
# save to excel
#new_df.to_excel("new_df.xlsx", index=False)

In [226]:
# filtered_df = filtered_df[['cumulative_import', 'cumulative_export', 'rate1', 'rate2', 'rate3','rate4','rate5','rate6' ]]
# filtered_df = filtered_df.astype(float)
# filtered_df = filtered_df.diff().fillna(0)
# filtered_df

In [231]:
from datetime import datetime
import calendar

# Given datetime string
date_string = "2025-01-17 00:30"
date_obj = datetime.strptime(date_string, "%Y-%m-%d %H:%M")

# Get the first day of the month (which is just the 1st)
start_of_month = date_obj.replace(day=1)

# Get the last day of the month
end_of_month = datetime(date_obj.year, date_obj.month, calendar.monthrange(date_obj.year, date_obj.month)[1])

# Check if the date is within the range
is_in_range = start_of_month <= date_obj <= end_of_month

print(is_in_range)  # Output: True


True
